## Load models

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cuda")

model_name = {
  "phi-2": "microsoft/phi-2",
  "phi-3": "microsoft/Phi-3-mini-4k-instruct", 
  "mistral": "mistralai/Mistral-7B-v0.3"
}
model = AutoModelForCausalLM.from_pretrained(model_name["phi-2"], torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name["phi-2"], trust_remote_code=True)

/users/abdffsm/miniconda/envs/llm-pred/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.32s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Prepare examples and Generate

In [22]:
import pandas as pd
import numpy as np

# numpy random generator 
random_generator = np.random.default_rng()

# Load the CSV file into a DataFrame
timing_data = pd.read_csv('../data/timing_data.csv')

# filter by model name 
df_gpt3 = timing_data[timing_data['model'].str.contains('gpt3')]
df_gpt4 = timing_data[timing_data['model'].str.contains('gpt-4')]
df_gpt4_new = timing_data[timing_data['model'].str.contains('gpt4-new')]
df_claude_opus = timing_data[timing_data['model'].str.contains('claude-3-opus')]
df_claude_sonnet = timing_data[timing_data['model'].str.contains('claude-3-sonnet')] 
df_claude_haiku = timing_data[timing_data['model'].str.contains('claude-3-haiku')] 

dataframes = [df_gpt3, df_gpt4, df_gpt4_new, df_claude_opus, df_claude_sonnet, df_claude_haiku]

for df_index, df in enumerate(dataframes):
    # eliminate outliers
    dataframes[df_index] = df[df['time_taken (s)'] < 1000]

# Randomly select samples
examples = []
example_num = 10
df_gpt3_sample = df_gpt3.sample(n=example_num+1, random_state=random_generator)
# Get the last row of the DataFrame
df_gpt3_input = df_gpt3_sample.iloc[-1]
# Get all the rows but the last one
df_gpt3_example = df_gpt3_sample.iloc[:-1]
# Iterate through the DataFrame with index starting from 0 
for index, data in enumerate(df_gpt3_example.iterrows()):
    current_example = ""
    current_example += "Prompt" + str(index + 1) + ": " + data[1]['prompt'] + "\n\n\n"
    current_example += "GPT Response" + str(index + 1) + ": " + data[1]['response'] + "\n\n\n"
    examples.append(current_example)

sys_prompt = """Given a prompt, respond in the same length as GPT-3.5 does. \n\n\n"""
example_prompt = """Here are some examples. Each example consists of a prompt and a response. \n\n\n"""
for example in examples:
    example_prompt += example
input = """Now, the prompt for you to predict is: """ + df_gpt3_input['prompt'] + "\n\n\n"
prompt = sys_prompt + example_prompt + input
inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_new_tokens = 1000)
text = tokenizer.batch_decode(outputs)[0]
print(text)

print ("\n\n\n --------------- \n\n\n")

print("Ground truth: \n" + df_gpt3_input['response'])

Token indices sequence length is longer than the specified maximum sequence length for this model (2253 > 2048). Running this sequence through the model will result in indexing errors
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Given a prompt, respond in the same length as GPT-3.5 does. 


Here are some examples. Each example consists of a prompt and a response. 


Prompt1: What is the growth rate in agriculture and IT sectors in Pakistan?


GPT Response1: As of the most recent data, Pakistan has experienced a significant growth rate in both the agriculture 🌾 and IT sectors 💻. The agriculture sector has shown steady growth due to various government initiatives and technological advancements, contributing to the country's GDP. On the other hand, the IT sector has been rapidly expanding, with a growing number of startups, software companies, and freelancers making their mark on the global stage. These developments reflect positive progress in Pakistan's economy, paving the way for a bright future in these crucial sectors.


Prompt2: The query is: How are you feeling? and the original response is: I'm feeling great, thanks for asking! Just here, ready to assist and chat about anything you need. How about you? Ho

## Pull data and export spreadsheets

In [ ]:
import pandas